Google Colab Notebook for Training Transformers

In [1]:
!pip install transformers

In [2]:
import torch
from torch import nn
from transformers import ElectraModel
# from transformers import ElectraTokenizer
from transformers import BertTokenizer, BertModel

### BERT and Electra use the same tokenizer
# ELECTRA_TOKENIZER = ElectraTokenizer.from_pretrained('google/electra-small-discriminator')
# ELECTRA_BASE_TOKENIZER = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')
BERT_TOKENIZER = BertTokenizer.from_pretrained('bert-base-uncased')

class Electra(nn.Module):
    def __init__(self, output_size: int, size = 'small', device='cpu'):
        super().__init__()
        self.device = device
        self.model = ElectraModel.from_pretrained(f'google/electra-{size}-discriminator').to(device)
        self.output = nn.Linear(self.model.config.hidden_size, output_size).to(device)

    # What happens when passing input into the model.
    def forward(self, sents, locs):
        sents = self.model(sents)[0]
        abbs = torch.stack([sents[n, idx, :] for n, idx in enumerate(locs)]) 
        return self.output(abbs)

class Bert(nn.Module):
    def __init__(self, output_size, device='cpu'):
        super().__init__()
        self.device = device
        self.model = BertModel.from_pretrained('bert-base-uncased').to(device)
        self.output = nn.Linear(self.model.config.hidden_size, output_size).to(device)

    # What happens when passing input into the model.
    def forward(self, sents, locs):
        sents = self.model(sents)[0]
        abbs = torch.stack([sents[n, idx, :] for n, idx in enumerate(locs)])
        return self.output(abbs)




In [3]:
torch.cuda.is_available()

True

In [16]:
import numpy as np

class MedalDatasetTokenizer(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, dictionary_file, max_length=256, device='cpu'):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.device = device
        self.df = df
        label_df = pd.read_csv(dictionary_file, sep='\t', index_col = "EXPANSION")
        self.label_ser = label_df["LABEL"].squeeze()


    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, idxs):
        
        batch_df = self.df.iloc[idxs]
        
        ### Locations used in MeDAL paper
        locs = batch_df['LOCATION'].values

        ### Locations of tokens corresponding to abbreviations
        # locs = batch_df['LONG_LOC'].values+1

        # locs = np.zeros(batch_df['LOCATION'].size, dtype = int)
        label_strings = batch_df['LABEL'].values
        labels = self.label_ser[label_strings].to_numpy()

        batch_encode = self.tokenizer(batch_df['TEXT'].tolist(), max_length=self.max_length, \
                    padding=True, truncation = True)

        tokenized = batch_encode['input_ids']
        return torch.tensor(tokenized).to(self.device), torch.tensor(locs).to(self.device), \
            torch.tensor(labels).to(self.device)

In [5]:
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np
from datetime import datetime
import os
from time import time 
import pandas as pd

In [19]:
def train_loop(train_data, model, loss_fn, optimizer, train_loader, max = -1):
    
    size = len(train_data)

    # Switches model to training mode.
    model.train()
    
    # Stores loss at each step.
    loss_list = []

    # For computing accuracy
    correct = 0

    for batch, idx in enumerate(tqdm(train_loader)):

        X = train_data[idx][0]
        loc = train_data[idx][1]
        y = train_data[idx][2]

        # Compute prediction and loss
        pred = model(X, loc)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Record the loss and accuracy
        loss_value = loss.item()
        batch_correct = (pred.argmax(1) == y).type(torch.float).sum().item()
        correct += batch_correct
        loss_list.append(loss_value)

        # Terminate early for testing purposes
        if max > 0:
            if batch >= max:
                print("\nMax iterations reached.")
                break

        # Periodically print the loss
        if batch % 100 == 0 and batch != 0:
            print(f"\nBatch loss: {loss_value:>7f}")
    
    loss_list = np.array(loss_list)
    mean_loss = np.mean(loss_list)
    accuracy = correct/size
    print(f"Accuracy: {accuracy:>3f} | Average Loss: {mean_loss:>7f}\n")
    
    return mean_loss, accuracy

# Tests the model on the validation or test data
def valid_loop(valid_data, model, loss_fn, valid_loader, max = -1):

    # Switches model to evaluation mode
    model.eval()

    size = len(valid_data)
    loss_list = [] 
    correct = 0

    with torch.no_grad():
        for batch, idx in tqdm(enumerate(valid_loader)):
        # for batch, idx in enumerate(valid_loader):

            X = valid_data[idx][0]
            loc = valid_data[idx][1]
            y = valid_data[idx][2]

            pred = model(X, loc)

            loss_list.append(loss_fn(pred, y).item())
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

            if max > 0:
                if batch > max:
                    break

    valid_loss = np.mean(np.array(loss_list))
    correct /= size
    print(f"Validation| \nAccuracy: {correct:>3f} | Average loss: {valid_loss:>8f} \n")
    
    return valid_loss, correct

In [7]:
# Save the model's state_dict in its current state. 
# Saved file name records current time and epoch number
def save_model(model, save_dir):
    now = datetime.now()
    time_formatted = now.strftime("%d")+"_"+now.strftime("%H")+"_"+now.strftime("%M")
    torch.save(model.state_dict(), save_dir + f"_{time_formatted}_StateDict.pt")
    print("Model saved\n")

In [17]:
###
### Train the model
###

# Use a GPU if available
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu" 

### Use a CPU for more clear error messages.
# dev = "cpu"

device = torch.device(dev)

# Set max to a positive integer to terminate training early.
# For debuggin purposes.
max = -1

tokenizer = BERT_TOKENIZER

# Data
num_abbr = "two_abbr"
folder = "drive/MyDrive/Bootcamp"
dictionary_file = f"{folder}/{num_abbr}/dict.txt"

train_df = pd.read_csv(f"{folder}/{num_abbr}/train_long_loc.csv")
train_data = MedalDatasetTokenizer(train_df, tokenizer, dictionary_file, device = device)

valid_df = pd.read_csv(f"{folder}/{num_abbr}/valid_long_loc.csv")
valid_data = MedalDatasetTokenizer(valid_df, tokenizer, dictionary_file, device = device)

# Hyperparameters
learning_rate = 4e-5 # Trains faster with similar acc as original 2e-5
batch_size = 16
epochs = 15

### Models 
output_size = 25 # Should be set to the size of the dictionary

# model = Bert(output_size, device)
model = Electra(output_size=output_size, size = 'small', device=device)

### Load a saved model. The correct model above must be initialized.
# path = f"{folder}/saves/eight_abbr_epoch3_27_23_21_StateDict.pt"
# model.load_state_dict(torch.load(path))

optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()

# Train the model
for t in range(epochs):
    print(f"\nEpoch {t+1}\n-------------------------------")
    
    # Randomly breaks the data into batches
    train_loader = DataLoader(
        range(len(train_data)), 
        shuffle=True, 
        batch_size=batch_size
    )
    valid_loader = DataLoader(
        range(len(valid_data)), 
        shuffle=True, 
        batch_size=batch_size
    )

    start = time()
    train_loss, train_accuracy = train_loop(train_data, model, loss_fn, optimizer, train_loader, max = max)
    end = time()
    print(f"Training time: {end-start:>0.1f} sec\n")

    valid_loss, valid_accuracy = valid_loop(valid_data, model, loss_fn, valid_loader, max = max)

    # Write the loss and accuracy to a file
    with open(f"{folder}/saves/loss.txt", "a") as file:
        file.writelines(f"\n{t+1},{train_loss},{train_accuracy},{valid_loss},{valid_accuracy}")

    save_model(model, f"{folder}/saves/{num_abbr}_epoch{t+1}")

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
  0%|          | 0/294 [00:00<?, ?it/s]


Epoch 1
-------------------------------


 34%|███▍      | 101/294 [00:36<01:08,  2.80it/s]


Batch loss: 2.906245


 68%|██████▊   | 201/294 [01:12<00:33,  2.75it/s]

...


Batch loss: 0.048343


100%|██████████| 294/294 [01:44<00:00,  2.82it/s]
0it [00:00, ?it/s]

Accuracy: 0.986161 | Average Loss: 0.056378

Training time: 104.2 sec



97it [00:27,  3.52it/s]


Validation| 
Accuracy: 0.857881 | Average loss: 0.576169 

Model saved



In [20]:
###
### Test the model
###

# Use a GPU if available
if torch.cuda.is_available():  
    dev = "cuda:0" 
else:  
    dev = "cpu" 
### Use a CPU for more clear error messages.
# dev = "cpu"
device = torch.device(dev)

# Data
num_abbr = "two_abbr"
folder = "drive/MyDrive/Bootcamp"
dictionary_file = f"{folder}/{num_abbr}/dict.txt"

tokenizer = BERT_TOKENIZER
test_df = pd.read_csv(f"{folder}/{num_abbr}/test_long_loc.csv")
test_data = MedalDatasetTokenizer(test_df, tokenizer, dictionary_file, device = device)

### Models 
output_size = 25 # Should be set to the size of the dictionary

#model = Bert(output_size, device)
model = Electra(output_size=output_size, size = 'small', device=device)
#model = Electra(output_size=output_size, size = 'base', device=device)

### Load a saved model. The correct model above must be initialized.
path = f"{folder}/saves/Finished/ElectraSmall_TwoAbbr_OldLoc_Epoch12.pt"
model.load_state_dict(torch.load(path))
#model = torch.load(path)

batch_size = 16
test_loader = DataLoader(
        range(len(test_data)), 
        shuffle=True, 
        batch_size=batch_size
    )

loss_fn = nn.CrossEntropyLoss()
test_loss, test_accuracy = valid_loop(test_data, model, loss_fn, test_loader)
test_loss, test_accuracy

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
98it [00:28,  3.49it/s]

Validation| 
Accuracy: 0.881302 | Average loss: 0.460943 



(0.4609425602984444, 0.8813018506700702)